## 나이브 베이지안

In [ ]:
전체 = 50
전체 중 스팸  = 15
전체 메일 중 로또  = 30
스팸 메일 중 로또  = 10
p(스팸) = 15/50
p(로또|메일) = 30/50
p(로또|스팸) = 10/15

p(스팸|로또) = p(로또|스팸)*p(스팸)/p(로또)

In [3]:
a = ((10/15)*(15/50))/0.6
a

0.3333333333333333

In [ ]:
전체 = 50
전체 중 스팸  = 15
전체 메일 중 로또  = 30
스팸 메일 중 로또  = 10
p(스팸) = 15/50
p(로또|메일) = 30/50
p(로또|스팸) = 10/15
p(경품|메일) = 10/50
p(경품|스팸) = 7/15

p(스팸|로또, 경품) = (p(로또,경품|스팸)*p(스팸))/p(로또,경품)
                   = (p(로또|스팸)*p(경품|스팸)*p(스팸))/(p(스팸|로또,경품)+p(스팸아님|로또,경품)) 

## 직접 구현

In [ ]:
## 긍정(1), 부정(0)
## 새로운 메일이 왔는데 happy weekend가 포함되어 있을 경우, 긍정인지 부정인지?

In [119]:
import pandas as pd
from collections import defaultdict
import numpy as np
df = pd.read_csv('naivebayes_example.csv')
df

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [120]:
## 생각
## 긍정, 부정을 나눠서 생각하기
## 나중에 병합하기

docs = df[df['label']==1]
docs = docs['mail'].tolist()

In [121]:
doc_ls = []
for doc in docs :
    doc_ls.append(doc.split(' '))
doc_ls

[['i', 'love', 'you'], ['love', 'happy', 'weekend'], ['happy', 'together']]

In [122]:
word2id  = defaultdict(lambda : len(word2id))

for doc in doc_ls :
    for token in doc :
        word2id[token]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'i': 0,
             'love': 1,
             'you': 2,
             'happy': 3,
             'weekend': 4,
             'together': 5})

In [123]:
TDM = np.zeros((len(word2id), len(doc_ls)), dtype = int)

for i, doc in enumerate(doc_ls) :
    for token in doc :
        TDM[word2id[token],i] += 1
TDM

array([[1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 1, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [124]:
po_ls = []
for i in TDM :
    po_ls.append(sum(i))
po_ls

[1, 2, 1, 2, 1, 1]

In [125]:
df1 = pd.DataFrame(word2id.keys())
df1.rename({0:'토큰 분류'}, axis = 1)


df1['positive'] = po_ls
df1

,0,positive
0,i,1
1,love,2
2,you,1
3,happy,2
4,weekend,1
5,together,1


In [126]:
## 부정
docs = df[df['label']==0]
docs = docs['mail'].tolist()

In [127]:
doc_ls = []
for doc in docs :
    doc_ls.append(doc.split(' '))
doc_ls

[['bore', 'work', 'job'], ['i', 'hate', 'you'], ['bore', 'weekend']]

In [128]:
word2id  = defaultdict(lambda : len(word2id))

for doc in doc_ls :
    for token in doc :
        word2id[token]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'bore': 0,
             'work': 1,
             'job': 2,
             'i': 3,
             'hate': 4,
             'you': 5,
             'weekend': 6})

In [129]:
TDM = np.zeros((len(word2id), len(doc_ls)), dtype = int)

for i, doc in enumerate(doc_ls) :
    for token in doc :
        TDM[word2id[token],i] += 1
TDM

array([[1, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [130]:
ne_ls = []
for i in TDM :
    ne_ls.append(sum(i))
ne_ls

[2, 1, 1, 1, 1, 1, 1]

In [131]:
df2 = pd.DataFrame(word2id.keys())
df2.rename({0:'토큰 분류'}, axis = 1)


df2['positive'] = ne_ls
df2

,0,positive
0,bore,2
1,work,1
2,job,1
3,i,1
4,hate,1
5,you,1
6,weekend,1


In [137]:
data = pd.merge(df1,df2, on = 0,how = 'outer')
data = data.fillna(int(0))
data

,0,positive_x,positive_y
0,i,1.0,1.0
1,love,2.0,0.0
2,you,1.0,1.0
3,happy,2.0,0.0
4,weekend,1.0,1.0
5,together,1.0,0.0
6,bore,0.0,2.0
7,work,0.0,1.0
8,job,0.0,1.0
9,hate,0.0,1.0


In [143]:
def po(po,ne) :
    up = po + 1
    down = sum(data['positive_x']) + len(data)
    a = up/down
    return a

data['P'] = data.apply(lambda x:po(x.positive_x, x.positive_y), axis = 1)
data

,0,positive_x,positive_y,P,N
0,i,1.0,1.0,0.111111,0.111111
1,love,2.0,0.0,0.166667,0.111111
2,you,1.0,1.0,0.111111,0.111111
3,happy,2.0,0.0,0.166667,0.111111
4,weekend,1.0,1.0,0.111111,0.111111
5,together,1.0,0.0,0.111111,0.055556
6,bore,0.0,2.0,0.055556,0.111111
7,work,0.0,1.0,0.055556,0.055556
8,job,0.0,1.0,0.055556,0.055556
9,hate,0.0,1.0,0.055556,0.055556


In [181]:
def po(po,ne) :
    up = ne+1
    down = sum(data['positive_y']) + len(data)
    a = up/down
    return a

data['N'] = data.apply(lambda x:po(x.positive_x, x.positive_y), axis = 1)
data

,positive_x,positive_y,P,N
0,,,,
i,1.0,1.0,0.111111,0.111111
love,2.0,0.0,0.166667,0.055556
you,1.0,1.0,0.111111,0.111111
happy,2.0,0.0,0.166667,0.055556
weekend,1.0,1.0,0.111111,0.111111
together,1.0,0.0,0.111111,0.055556
bore,0.0,2.0,0.055556,0.166667
work,0.0,1.0,0.055556,0.111111
job,0.0,1.0,0.055556,0.111111


In [195]:
## exp랑 log추가해보기

def positive(a,b) :
    a_po = data.loc[a].loc['P']
    b_po = data.loc[b].loc['P']
    p_po = 0.5
    a_ne = data.loc[a].loc['N']
    b_ne = data.loc[b].loc['N']
    p_ne = 0.5
    
    result = (a_po*b_po*p_po)/((a_po*b_po*p_po)+(a_ne*b_ne*p_ne))
    return result

positive = positive('happy', 'weekend')
print(positive)

negative = 1-positive
print(negative)

0.75
0.25


## Sklearn으로 구현하기

In [196]:
X_train = list(df['mail'])
Y_train = list(df['label'])

In [197]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
## fit and transform
X_train_counts = count_vect.fit_transform(X_train)
## MultinomialNB 선언 및 fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [198]:
## 예측
print(clf.predict(count_vect.transform(['happy weekend'])))

## 확률
print(clf.predict_proba(count_vect.transform(['happy weekend'])))

[1]
[[0.25 0.75]]


## 네이버 영화 리뷰 감성분석

In [375]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('./ratings_train.txt', sep='\t', quoting=3))
df = df.replace(np.nan, '', regex=True)
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [376]:
import re 
# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
def preprocess(text):
    text = re.sub(remove_except_ko,' ',text).strip()
    return text

df['document'] = df['document'].map(lambda x : preprocess(x))

In [377]:
from konlpy.tag import Mecab
mecab = Mecab()
# mecab을 사용해서 토큰화, 한글자 제거, stop_word 제거 
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']

# def token_mecab(text):
#     text = mecab.morphs(text)
#     text = [i for i in text if len(i)>1]
#     text = [i for i in text if i not in stop_word]
#     return text

stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']
def token_mecab(text):
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])>1]
    text = [i for i in text if i[0] not in stop_word]
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [378]:
df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [379]:
# 토큰화 + 토큰리스트 생성 
def make_tokens(df):
    df['tokens'] = ''
    tokens_list=[]
    for i, row in df.iterrows():
        if i%10000==0:
            print(i,'/',len(df))
        token = token_mecab(df['document'][i])
        df['tokens'][i] = ' '.join(token)
    return df

df = make_tokens(df)

0 / 150000


C:\Users\user\.conda\envs\py37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 150000
20000 / 150000
30000 / 150000
40000 / 150000
50000 / 150000
60000 / 150000
70000 / 150000
80000 / 150000
90000 / 150000
100000 / 150000
110000 / 150000
120000 / 150000
130000 / 150000
140000 / 150000


In [380]:
df.head()

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,짜증 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,포스터 초딩 영화 오버 연기 가볍 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,교도소 이야기 구먼 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 보이 커스틴 던스트 이뻐 보였


In [381]:
X_train = list(df['tokens'])
Y_train = list(df['label'])

In [382]:
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [383]:
test = '강력한 몰입감을 선사하는 두 배우의 연기력!! 오랜만에 정말 색다른 영화였다'
test = token_mecab(test)
test = ' '.join(test)
print(test)
# 예측
print(clf.predict(count_vect.transform([test]))) 
# 확률 
print(clf.predict_proba(count_vect.transform([test])))

강력 입감 선사 배우 연기력 오랜만 색다른 영화
[1]
[[0.00266325 0.99733675]]


## 예측 결과 정확도 측정

In [387]:
import pandas as pd 
test_df = pd.DataFrame(pd.read_csv('./ratings_test.txt', sep='\t', quoting=3))
test_df = test_df.replace(np.nan, '', regex=True)
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [388]:
test_df['document'] = test_df['document'].map(lambda x : preprocess(x))

In [389]:
test_df = make_tokens(test_df)
test_df.head()

0 / 50000


C:\Users\user\.conda\envs\py37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 50000
20000 / 50000
30000 / 50000
40000 / 50000


,id,document,label,tokens
0,6270596,굳 ㅋ,1,
1,9274899,,0,
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편


In [390]:
predic_label = []
for i, row in test_df.iterrows():
    predic = clf.predict(count_vect.transform([test_df['tokens'][i]]))[0]
    predic_label.append(predic)
test_df['predic_label'] = predic_label
test_df.head()

,id,document,label,tokens,predic_label
0,6270596,굳 ㅋ,1,,0
1,9274899,,0,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편,0


#### Confusion Matrix

In [391]:
TP = FN = FP = TN = other = 0
for i in range(len(test_df)):
    if test_df['label'][i] == 1 and test_df['predic_label'][i] == 1:
        TP +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 0:
        TN +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 1:
        FN +=1
    elif test_df['label'][i] == 1 and test_df['predic_label'][i] == 0:
        FP +=1
    else:
        other +=1
print(TP,TN,FN,FP)

19632 20393 4434 5541


In [392]:
ACC = (TP+TN)/(TP+TN+FP+FN)
print('ACC: ',ACC)

ACC:  0.8005
